# VICA Technical Assessment - Task 2

Name of candidate: Chan Choon Kong

## Import libraries

In [1]:
# !pip install pymongo
import os
import pymongo
import pandas as pd
import numpy as np
from pymongo import MongoClient
from collections import OrderedDict

## Set up database

Steps taken to set up MongoDB locally (Windows):
1. Download MongoDB from https://www.mongodb.com/try/download/community
2. Install MongoDB
3. Add `C:\Program Files\MongoDB\Server\6.0\bin` to PATH
4. Create a folder for MongoDB data files
5. Open command prompt and run the command `mongod --dbpath <path-to-folder>`

### Making a connection with MongoClient

In [2]:
client = MongoClient('localhost', 27017)
db = client['test']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')

### Create collection for data

In [3]:
if "insuranceData" not in db.list_collection_names():
    db.create_collection("insuranceData")
db.list_collection_names()

['insuranceData']

### Define schema validation rules

The schema follows the data structure specified in page 2 of the PDF file. The descriptions are also directly copied from the PDF. Columns without missing values are all required entries in the input data. Columns with missing values are not required to avoid introducing unnecessary null values in the dataset.

The fields `gender` and `premiumFrequency` are defined as enum types as they are non-boolean categorical variables. Similarly, the values within `riders` are also enum types as there are only 4 riders in the given dataset. In the event that any of these variables can take on more discrete values, the schema would have to be updated.

In [4]:
schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["insuree#", "isMarried", "hasKids", "insuredMonths", "termLifeInsurance", "healthInsurance", "eStatements", "monthlyPremium", "renewal"],
        "properties": {
            "insuree#": {
                "bsonType": "int",
                "description": "ID  of the insuree"
            },
            "gender": {
                "enum": ["M", "F"],
                "description": "M or F"
            },
            "is45OrOlder": {
                "bsonType": "bool",
                "description": "Is insuree >= 45 years old?"
            },
            "isMarried": {
                "bsonType": "bool",
                "description": "Is insuree married?"
            },
            "hasKids": {
                "bsonType": "bool",
                "description": "Does the insuree have kids?"
            },
            "insuredMonths": {
                "bsonType": "int",
                "description": "Months of active insurance"
            },
            "termLifeInsurance": {
                "bsonType": "object",
                "required": ["hasPolicy", "hasMultiplePolicies"],
                "properties": {
                    "hasPolicy": {
                        "bsonType": "bool",
                        "description": "Does insuree have term life policy?"
                    },
                    "hasMultiplePolicies": {
                        "bsonType": "bool",
                        "description": "Does insuree hold multiple term life policies?"
                    }
                }
            },
            "healthInsurance": {
                "bsonType": "object",
                "required": ["hasPolicy", "riders"],
                "properties": {
                    "hasPolicy": {
                        "bsonType": "bool",
                        "description": "Does insuree have health insurance?"
                    },
                    "riders": {
                        "bsonType": "array",
                        "items": {
                            "enum": [1, 2, 3, 4]
                        },
                        "description": "Does insuree have riders on this health policy?"
                    }
                }
            },
            "premiumFrequency": {
                "enum": [1, 3, 12],
                "description": "Premium due monthly, quarterly, annually"
            },
            "eStatements": {
                "bsonType": "bool",
                "description": "Opted in for e-statements and e-policies?"
            },
            "monthlyPremium": {
                "bsonType": "double",
                "description": "Premium amount monthly"
            },
            "totalPremium": {
                "bsonType": "double",
                "description": "Total premium amount"
            },
            "renewal": {
                "bsonType": "bool",
                "description": "Does insuree renew at next premium cycle?"
            }
        }
    }
}

cmd = OrderedDict([('collMod', 'insuranceData'), ('validator', schema)])
if "insuranceData" not in db.list_collection_names():
    db.command(cmd)

The output is `{'ok': 1.0}` which indicates that execution of the command worked.

## Load dataset

In [5]:
dataset_dir = os.path.join(os.curdir, 'mol-vica-ds-challenge-dataset', 'insurance_data.csv')

# Define converters
float_to_bool = lambda x: x == '1.0' if x else None
yesno_to_bool = lambda x: (x == 'Yes' or x == 'Y') if x else None
csfloat_to_dec = lambda x: float(x.replace(',', '.')) if x.replace(',', '.').strip() else np.nan
csint_to_lst = lambda x: list(map(lambda y: int(y), x.split(','))) if x.split(',')[0] else []

df = pd.read_csv(dataset_dir, sep=';', converters={
    'is45OrOlder': float_to_bool, 
    'isMarried': yesno_to_bool, 
    'hasKids': yesno_to_bool, 
    'termLifeInsurance': yesno_to_bool, 
    'multipleTermLifePolicies': yesno_to_bool,
    'eStatements': yesno_to_bool,
    'renewal': yesno_to_bool,
    'healthRiders': csint_to_lst
})

# Note: Int64 writes <NA> for empty integer fields
df['premiumFrequency'] = df['premiumFrequency'].astype('Int64')
df['monthlyPremium'] = df['monthlyPremium'].apply(csfloat_to_dec)
df['totalPremium'] = df['totalPremium'].apply(csfloat_to_dec)
df.head(5)

,insuree#,gender,is45OrOlder,isMarried,hasKids,insuredMonths,termLifeInsurance,multipleTermLifePolicies,healthInsurance,healthRiders,premiumFrequency,eStatements,monthlyPremium,totalPremium,renewal
0,1,F,False,True,True,23,True,False,No,[],12,True,19.65,451.55,True
1,2,F,True,False,False,42,True,True,Class A,[3],1,True,84.65,3541.35,False
2,3,F,False,True,False,72,True,False,No,[],12,False,19.40,1496.45,True
3,4,F,False,True,True,13,True,False,No,[],12,False,19.55,265.30,True
4,5,F,False,False,False,37,True,True,Class A,"[3, 4]",1,False,100.30,3541.40,True


In [6]:
cols_missing = df.columns[df.isnull().any()].tolist()
print("Columns with missing inputs: {0}".format(cols_missing))

Columns with missing inputs: ['gender', 'is45OrOlder', 'premiumFrequency', 'totalPremium']


## Push data into MongoDB

Data from the dataframe is pushed into the collection one row at a time. This is preferred over pushing the entire dataframe at once so as to reduce memory consumption.

In [7]:
def process_row_record(dic):
    dic['termLifeInsurance'] = {
        'hasPolicy': dic['termLifeInsurance'],
        'hasMultiplePolicies': dic['multipleTermLifePolicies']
    }
    dic['healthInsurance'] = {
        'hasPolicy': dic['healthInsurance'] != 'No',
        'riders': dic['healthRiders']
    }

    del dic['multipleTermLifePolicies']
    del dic['healthRiders']
    for col in cols_missing:
        if pd.isna(dic[col]):
            del dic[col]
    return dic

In [8]:
if "insuranceData" not in db.list_collection_names():
    for dic in df.to_dict(orient='records'):
        dic = process_row_record(dic)
        # Insert into database
        db.insuranceData.insert_one(dic)

Now, I will run some queries to check that the data has been successfully pushed into the `insuranceData` collection in the database.

In [9]:
db.insuranceData.find_one({'premiumFrequency': None})

{'_id': ObjectId('62f65c4cde12319ef35fc315'),
 'insuree#': 21,
 'gender': 'F',
 'is45OrOlder': True,
 'isMarried': False,
 'hasKids': False,
 'insuredMonths': 4,
 'termLifeInsurance': {'hasPolicy': True, 'hasMultiplePolicies': True},
 'healthInsurance': {'hasPolicy': True, 'riders': []},
 'eStatements': True,
 'monthlyPremium': 74.45,
 'totalPremium': 294.45,
 'renewal': True}

## Data Query

Now, I will query the database to find answers to the guiding questions in the PDF. I assume that preprocessing (e.g. handling outliers, missing values, etc.) is not required here as only the original data is stored in the database, and having to do preprocessing after each query is not efficient. For results after preprocessing the dataset, refer to the Analysis of Riders section in Task 1.

1. What is the average number of riders health insurance customers purchase, and which ones are the most popular?

In [25]:
pipeline = [
    {"$match": {"healthInsurance.hasPolicy": True}},
    {"$project": {"count": {"$size": "$healthInsurance.riders"}}}
]
num_riders = list(map(lambda x: x['count'], db.insuranceData.aggregate(pipeline)))
average_riders = sum(num_riders) / len(num_riders)
print("Average number of riders health insurance customers purchase: {0}".format(round(average_riders, 2)))

Average number of riders health insurance customers purchase: 1.78


In [41]:
from bson.son import SON
pipeline = [
    {"$match": {"healthInsurance.hasPolicy": True}},
    {"$unwind": "$healthInsurance.riders"},
    {"$group": {"_id": "$healthInsurance.riders", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1), ("_id", -1)])}
]
results = list(db.insuranceData.aggregate(pipeline))
total_count = sum(map(lambda x: x['count'], results))
for res in list(results):
    rider = res['_id']
    count = res['count']
    print("Rider {}: {} ({:.1f}%)".format(rider, count, 100*count/total_count))

Rider 4: 2101 (27.5%)
Rider 3: 2099 (27.4%)
Rider 2: 1888 (24.7%)
Rider 1: 1564 (20.4%)


As we can see, rider 4 is the most popular rider with 2101 health insurance customers having it, while rider 3 is the second most popular rider with 2099 health insurance customers having it.

2. What is the most popular rider for older people?

In [42]:
pipeline = [
    {"$match": {"is45OrOlder": True}},
    {"$unwind": "$healthInsurance.riders"},
    {"$group": {"_id": "$healthInsurance.riders", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1), ("_id", -1)])}
]
results = list(db.insuranceData.aggregate(pipeline))
total_count = sum(map(lambda x: x['count'], results))
for res in list(results):
    rider = res['_id']
    count = res['count']
    print("Rider {}: {} ({:.1f}%)".format(rider, count, 100*count/total_count))

Rider 4: 437 (30.2%)
Rider 3: 429 (29.6%)
Rider 2: 363 (25.1%)
Rider 1: 218 (15.1%)


The most popular rider for older people is also rider 4. Note that the results obtained here are similar to the results in Task 1 under the section Analysis of Riders.

## Considerations

In Task 1, the entire dataset was loaded onto memory (RAM) while in Task 2, the dataset was loaded onto the database. Since the dataset provided is relatively small (5500 rows of data), it can be loaded onto RAM for processing. However, if the dataset size was orders of magnitude larger, it may be too large to load onto RAM. In that case, loading onto the database before querying it will be the better solution.